In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
sns.set()

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import time
import re

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
df_aziende = pd.read_excel("imprese.xlsx",dtype={"CF ":'str',"Nominativo":"str"})
df_aziende

,CF,Nominativo
0,04427040284,innovie srl
1,02197660224,sintac srl
2,00230870222,menz&gasser spa
3,00122700222,trentofrutta spa
4,02215960226,bonfiglioli mechatronic research spa
...,...,...
192,04132640238,Veritre_srl
193,01683400228,Areaderma sas
194,02054990227,Areaderma srl
195,01308130226,Cortech_srl


In [3]:
lista_nominativi = list(df_aziende['Nominativo '])
lista_nominativi

['innovie srl ',
 'sintac srl ',
 'menz&gasser spa ',
 'trentofrutta spa',
 'bonfiglioli mechatronic research spa ',
 'business process engineering srl ',
 'Research & Development Systems Srl',
 'F.a.r. Systems spa ',
 'novabase srl ',
 'aurea center srl ',
 'dial funghi srl ',
 'XPeppers srl',
 'pama spa',
 'tamanini hydro srl ',
 'corehab srl ',
 'famatec spa',
 'italclem srl/spa',
 'mariani spa ',
 'trentinutensil srl ',
 'selecos srl ',
 'cora srl ',
 'tecnoclima spa ',
 'bv tech ricerca srl ',
 'tretec srl ',
 'marangoni spa ',
 'hsl srl ',
 'praim srl ',
 'non individuabile ',
 'lincotek trento spa',
 'dana transmission systems italia srl ',
 'etway srl ',
 'algorab srl ',
 'suggesto srl ',
 'GPI spa ',
 'edizioni centro studi erickson spa ',
 'telecom italia spa o tim spa ',
 'crg srl ( centro ricerche gpi srl)',
 'cogito srl ',
 'expert system spa ',
 'dedagroup spa ',
 'aldebra spa',
 'interplay software srl ',
 'okkam srl ',
 'spaziodati srl ',
 'Curcu & Genovese Associati sr

In [4]:
lista_codici_fiscali = list(df_aziende['CF '])
lista_codici_fiscali

['04427040284',
 '02197660224',
 '00230870222',
 '00122700222',
 '02215960226',
 '01886540226',
 '07704200588',
 '01289500223',
 '01775770223',
 '01856750227',
 '02439500220',
 '02181040227',
 '01766560229',
 '00317660223',
 '02185760226',
 '01219170220',
 '01161910177',
 '01208110229',
 '00316660224',
 '00649870227',
 '00496220229',
 '00199160227',
 '02049620228',
 '01837040227',
 '00125560227',
 '00649980224',
 '01613840220',
 '01623200225',
 '01305350223',
 '06262230961',
 '01971340227',
 '01367190228',
 '01938750229',
 '01944260221',
 '01063120222',
 '00488410010',
 '02211630229',
 '01865020224',
 '02608970360',
 '01763870225',
 '01170810228',
 '01904320221',
 '02185910227',
 '02241890223',
 '00587130220',
 '01976920221',
 '00424360220',
 '01945440228',
 '02183360227',
 '00143150225',
 '01474390224',
 '01713040226',
 '01429670225',
 '02031100221',
 '02243030224',
 '02249580222',
 '07084560015',
 '01112330228',
 '00641100227',
 '01414840221',
 '00120100227',
 '02261050229',
 '019739

In [12]:
lista_nominativi[82]

'clesius srl '

In [13]:
lista_codici_fiscali[82]

'01469760225'

In [5]:
pesca = []
service = Service(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
k=0
for cf in lista_codici_fiscali[83:]:
    dict_pesca = { "Codice fiscale":[],"Nominativo":[]}
    dict_pesca["Codice fiscale"].append(cf)
    dict_pesca["Nominativo"].append(lista_nominativi[k+83])
    k = k+1

    driver.get("https://tuttodati.it/")
    testo_cerca_home = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[2]/div/div[2]/div/form/input')
    testo_cerca_home.send_keys(str(cf)+Keys.ENTER)
    time.sleep(1)
    try:
        link_azienda = driver.find_element(by=By.XPATH, value="/html/body/div[1]/div[3]/table/tbody/tr[2]/td[1]/a")
    except:
        continue

    driver.get(link_azienda.get_attribute("href"))
    time.sleep(1)

    first_table = driver.find_element(by=By.XPATH,value="/html/body/div[1]/div[3]/div[2]/div[1]/div[1]/table")
    tabella1 = first_table.text.split("\n")
    for i in range(0,len(tabella1)-1,2):
        if tabella1[i] not in dict_pesca.keys():
            dict_pesca[tabella1[i]] = []
        dict_pesca[tabella1[i]].append(tabella1[i+1])

    try:
        second_table = driver.find_element(by=By.XPATH, value='/html/body/div[1]/div[3]/div[2]/div[1]/div[3]/div')
        tabella2 = second_table.text.split("\n")
    except:
        pesca.append(dict_pesca)
        continue

    for i in range(1,len(tabella2)-1,2):
        if tabella2[i]=="Utile":
            indice_stop = i
            break
        if tabella2[i] not in dict_pesca.keys():
            dict_pesca[tabella2[i]] = []
        dict_pesca[tabella2[i]].append(tabella2[i+1])
    
    for i in range(indice_stop+1,len(tabella2)-1,2):
        if tabella2[i] not in dict_pesca.keys():
            dict_pesca[tabella2[i]] = []
        dict_pesca[tabella2[i]].append(tabella2[i+1])

    pesca.append(dict_pesca)

driver.quit()



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [/home/davide/.wdm/drivers/chromedriver/linux64/100.0.4896.60/chromedriver] found in cache


In [6]:
pesca

[{'Codice fiscale': ['00866690225'],
  'Nominativo': ['rekord srl '],
  'Stato': ['In Liquidazione'],
  'Nome completo': ['REKORD SRL'],
  'P. IVA': ['01664954828'],
  'Regione': ['Trentino-Alto Adige'],
  'Città': ['ROVERETO'],
  'Indirizzo': ["VIA DELL'ARTIGIANO 57 (TN)"],
  'Visura camerale': ['Scarica']},
 {'Codice fiscale': ['01401010226'],
  'Nominativo': ['sg elettrica snc '],
  'Stato': ['In Liquidazione'],
  'Nome completo': ['SG ELETTRICA SNC DI CONDINI G. & ZATTARA S.'],
  'P. IVA': ['01564906844'],
  'Regione': ['Trentino-Alto Adige'],
  'Città': ['TRENTO'],
  'Indirizzo': ["LOCALITA' LE BASSE 6/3/4 MATTARELLO (TN)"],
  'Visura camerale': ['Scarica']},
 {'Codice fiscale': ['01769810225'],
  'Nominativo': ['logico team srl '],
  'Stato': ['Attiva'],
  'Nome completo': ['LOGICO TEAM S.R.L.'],
  'Codice ATECO': ["62.01: Produzione di software non connesso all'edizione"],
  'P. IVA': ['02605873446'],
  'Data di costituzione': ['23/11/2009'],
  'Regione': ['Trentino-Alto Adige']

In [7]:
df_tot = pd.DataFrame()
for d in pesca:
    for key in d:
        if len(d[key])==0:
            d[key].append(-1)
    df = pd.DataFrame(d)
    df_tot = pd.concat([df_tot,df],axis=0)
df_tot

,Codice fiscale,Nominativo,Stato,Nome completo,P. IVA,Regione,Città,Indirizzo,Visura camerale,Codice ATECO,...,Utile 2019,Utile 2020,Utile 2018 Perdita,Utile 2019 Perdita,Fatturato 2016,Utile 2016,Utile 2017 Perdita,Fatturato 2021,Utile 2021,Utile 2020 Perdita
0,00866690225,rekord srl,In Liquidazione,REKORD SRL,01664954828,Trentino-Alto Adige,ROVERETO,VIA DELL'ARTIGIANO 57 (TN),Scarica,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,01401010226,sg elettrica snc,In Liquidazione,SG ELETTRICA SNC DI CONDINI G. & ZATTARA S.,01564906844,Trentino-Alto Adige,TRENTO,LOCALITA' LE BASSE 6/3/4 MATTARELLO (TN),Scarica,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,01769810225,logico team srl,Attiva,LOGICO TEAM S.R.L.,02605873446,Trentino-Alto Adige,ROVERETO,"VIA PER MARCO, 12/B (TN)",Scarica,62.01: Produzione di software non connesso all...,...,€258K -95%,"€1,38M + 435%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,01095090229,edilpavimentazioni srl,Attiva,EDILPAVIMENTAZIONI S.R.L.,01326007889,Trentino-Alto Adige,LAVIS,"VIA BOLZANO, 2-4 (TN)",Scarica,"42.11: Costruzione di strade, autostrade e pis...",...,"€49,38M -55%","€69,02M + 39%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,02244380222,yonder srl,Attiva,YONDER S.R.L.,02399056216,Trentino-Alto Adige,TRENTO,"VIA PRAGA, 5 (TN)",Scarica,72.19.09: Ricerca e sviluppo sperimentale nel ...,...,€248K,€284K + 14%,€-12.924,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,04132640238,Veritre_srl,Attiva,VERITEC – S.R.L.,04543778613,Veneto,VERONA,"VIA ALBERE, 29/A (VR)",Scarica,43.21.01: Installazione di impianti elettrici ...,...,"€1,29M","€2,76M + 113%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,01683400228,Areaderma sas,In Liquidazione,AREADERMA S.A.S. LABORAT COSMET DI BIOPELS & C.,02323266201,Trentino-Alto Adige,PERGINE VALSUGANA,VIALE DELL' INDUSTRIA 47 (TN),Scarica,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,02054990227,Areaderma srl,Attiva,AREADERMA S.R.L.,02902765523,Trentino-Alto Adige,PERGINE VALSUGANA,"VIALE DELL'INDUSTRIA, 47 (TN)",Scarica,20.42: Fabbricazione di prodotti per toletta: ...,...,"€1,93M -85%","€14,25M + 637%",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,01308130226,Cortech_srl,In Liquidazione,CORTECH S R L,02049224820,Trentino-Alto Adige,CANAL SAN BOVO,LOCALITA' LAUSEN 13/A (TN),Scarica,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_tot = df_tot[df_tot["Codice fiscale"]==df_tot["P. IVA"]]

In [9]:
df_tot

,Codice fiscale,Nominativo,Stato,Nome completo,P. IVA,Regione,Città,Indirizzo,Visura camerale,Codice ATECO,...,Utile 2019,Utile 2020,Utile 2018 Perdita,Utile 2019 Perdita,Fatturato 2016,Utile 2016,Utile 2017 Perdita,Fatturato 2021,Utile 2021,Utile 2020 Perdita


In [17]:
df_tot.to_excel("tuttodati_new2.xlsx")